### Download the dataset
https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews    

In [6]:
DATA_PATH = "data-gpu-transformers/Scraped_Car_Review_dodge.csv"

import time
from tqdm import tqdm
import polars as pl
import torch
from transformers import (
    pipeline,
    TextClassificationPipeline,
)

In [7]:
reviews_list = pl.read_csv(DATA_PATH)["Review"].to_list()
len(reviews_list)


8499

In [8]:
reviews_list[0]

" It's been a great delivery vehicle for my cafe business good power, economy match easily taken care of. Havent repaired anything or replaced anything but tires and normal maintenance items. Upgraded tires to Michelin LX series helped fuel economy. Would buy another in a second"

In [9]:
def time_text_classifier(
    text_pipeline: TextClassificationPipeline,
    texts: list[str],
    batch_size: int = 1,
) -> None:
    """Time how long it takes a TextClassificationPipeline
       to run inference on a list of texts"""

    texts_generator = (t for t in texts)
    pipeline_iterable = tqdm(
        text_pipeline(
            texts_generator,
            batch_size=batch_size,
            truncation=True,
            max_length=500,
        ),
        total=len(texts),
    )

    for result in pipeline_iterable:
        pass


In [10]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_pipeline_cpu = pipeline(model=model_name, device=-1)
sentiment_pipeline_gpu = pipeline(model=model_name, device=0)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassifi

In [11]:
time_text_classifier(sentiment_pipeline_cpu, reviews_list[0:1000])

100%|██████████| 1000/1000 [01:07<00:00, 14.84it/s]


In [12]:
time_text_classifier(sentiment_pipeline_gpu, reviews_list[0:1000])

100%|██████████| 1000/1000 [02:34<00:00,  6.48it/s]


In [ ]:
batch_sizes = [1, 2, 4, 8, 10, 12, 15, 20, 50, 100]
for batch_size in batch_sizes:
    print(f"Batch size: {batch_size}")
    time_text_classifier(
        sentiment_pipeline_gpu,
        reviews_list,
        batch_size=batch_size
    )
